# Séance 1 - Réseau de neurones dense

On se propose de classifier les chiffres manuscrit du dataset [MNIST](https://yann.lecun.com/exdb/mnist/) en définissant ses propres réseaux de neurones denses. L'objectif est de découvrir la manière d'entraîner ces algorithmes et observer en pratique les bases théoriques discutées en cours.

## Exploration des données

Commençons par importer les données.

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')

import tensorflow as tf
from tensorflow import keras

(X_train_full, y_train_full), (X_test, y_test) = (keras.datasets.mnist.load_data())

**Consigne** : À l'aide de la fonction [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), séparer le jeu d'entraînement complet en un dataset d'entraînement et un dataset de validation. Afficher les tailles des datasets respectifs.

**Consigne** : Afficher plusieurs images du dataset d'entraînement aléatoirement. On pourra utiliser la fonction [`imshow`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html).

Les images sont en niveau de gris, donc de valeurs entre 0 et 255. Pour entraîner correctement un réseau de neurones, il est préférable que les inputs soit standardisés.

**Consigne** : Standardiser les données en utilisant la classe [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). On commencera par applatir les images en utilisant la méthode [`reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html), puis on applique le pré-processing et on termine par reformer la matrice.

## Modélisation du réseau de neurones

Pour le moment, nous travaillons avec des images de tailles $28\times28$, mais nous ne savons pas définir (pour le moment) de réseau de neurones capable de travailler directement avec une image. Nous allons utiliser une couche nommée [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) dont le but est *d'applatir* une matrice de dimension *(height, width, channel)* en un vecteur de taille *height $\times$ width $\times$ channel*. Dans le cadre des données MNIST, *channel*=1 puisque nous sommes en niveau de gris, et *height=width=28*. On aura un vecteur de 784 dimensions.

Une fois que nous aurons décrit l'ensemble du réseau, nous devrons terminer le réseau par une couche avec dix neurones : un pour chaque classe. Pour s'assurer que l'on aura une estimation de probabilité d'appartenance à la classe, on utilisera la fonction softmax. Pour un vecteur $x = (x_0, x_1, \ldots, x_n)$ on a:

$$\text{softmax}(x)_j = \frac{e^{x_j}}{\displaystyle \sum_{i=0}^n e^{x_i}}$$

On veut définir le réseau suivant:
* **Couche cachée 1** : 256 neurones avec fonction d'activation ReLU
* **Couche cachée 2** : 128 neurones avec fonction d'activation ReLU

On peut définir de plusieurs manières un réseau de neurones. La première est de la même manière qu'une liste à laquelle on ajoute des couches en utilisation le template de modèle *Sequential* :

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[28, 28]))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

En début de réseau de neurones on doit définir la dimension de l'input: ici (28, 28). Le reste des dimensions pour l'ensemble des couches qui lui succède sont calculées automatiquement.

La deuxième manière est directement sous le format d'une liste:

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[28, 28]),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

**Consigne** : Calculer à la main le nombre de neurones du modèle, couche par couche. Puis utiliser la méthode [`summary`](https://keras.io/api/models/model/#summary-method) pour vérifier les calculs.

Nous avons décrit l'architecture du réseau de neurones. Il faut maintenant définir comment il va s'entraîner. Nous devons spécifier:

* **Loss** : Quelle fonction de perte est à minimiser ?
* **Optimizer** Quel schéma de descente de gradient est à utiliser ?
* **Metrics** : Quelles métrique de performance souhaite-on observer pendant l'entraînement ?

Puisque nous travaillons sur un problème de classification avec plusieurs classes, la fonction de perte [`sparse_categorical_crossentropy`](https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class) est celle qu'il nous faut.

Concernant l'*optimizer* il y a plusieurs possibilités que nous verrons dans une prochaine séance. Pour le moment nous travaillerons avec une descente de gradient stochastique par mini-batch [`SGD`](https://keras.io/api/optimizers/sgd/). Pour la définir, nous devons statuer sur:
* **Learning rate** : pas de descente, on décide de choisir la valeur 0.001
* **Batch size** : nombre d'observations à considérer pour chacune des passes. On décide de prendre 32 images par batch. Cette valeur sera à renseigner un peu plus tard.

Pour les métriques, nous suivrons l'accuracy parce que la distribution des catégories à prédire n'est pas déséquilibrées.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1e-3), metrics=["accuracy"])

## Entraînement

Le modèle est maintenant prêt à être entraîné, il nous reste à lui indiquer:
* **Données** : jeu d'entraînement et jeu de validation
* **Époques** : le nombre de passes à réaliser sur l'ensemble du dataset
* **Batch size** : le nombre d'observations pour chaque batch, nous avions décidé juste avant que ce serait 32

In [ ]:
epochs = 5
batch_size = 32

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))

Nous avons des informations disponible dans l'objet *history*, plus précisement dans *history.history*

**Consigne** : Créer un [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) à partir de *history.history* puis inspecter-le.

**Consigne**: Définir une fonction `plot_learning_curves` qui prend en paramètre l'objet *history* et qui renvoie un graphique. Le graphique correspondra à deux graphiques côte à côte :
1. Le premier montre l'évolution de la fonction de perte en fonction des époques
2. Le second montre l'évolution de l'accuracy en fonction des époques
Dans les deux cas, les valeurs de performance sur le dataset de validation doivent être en pointillé. 

**Consigne** : Exploiter la fonction précédente pour observer les courbes d'apprentissage du l'entraînement précédent.

## Exploitation des prédictions

On souhaite à présent utiliser le modèle pour prédire le chiffre présent dans une image.

**Consigne** : Prédire sur le jeu de test à l'aide de la méthode [`predict`](https://keras.io/api/models/model_training_apis/#predict-method), puis observer le résultat sur la première image.

**Consigne** : Après avoir vérifier que si l'on somme les chiffres affichés pour la première image vaut bien 1, identifier la classe prédite par le modèle. Vérifier visuellement.

## Quel est l'impact du learning rate ?

On s'intéresse à présent à l'importance du choix du learning rate. On se propose de tester plusieurs valeurs pour obtenir les meilleurs performances.

**Consigne** : Définir une fonction `get_model` qui prend en paramètre un `float` qui correspond à un learning rate. La fonction renvoie un modèle compilé avec les mêmes paramètres que précédemment, sauf la valeur du learning rate qui est renseignée par l'utilisateur.

Avant de lancer sur plusieurs époques, commençons par écrire une ébauche de la boucle de comparaison avec 5 époques.

In [ ]:
n_epochs = 5
batch_size = 32
learning_rates = [10**(-power) for power in range(1, 4)]

results = []
for learning_rate in learning_rates:
  print("Learning rate: %f - époques: %d" % (learning_rate, n_epochs))
  model = get_model(learning_rate=learning_rate)
  history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))
  result = {"learning_rate": learning_rate, "n_epochs": n_epochs, "history": pd.DataFrame(history.history)}
  results.append(result)

**Consigne** : Définir une fonction `show_results` qui prend en paramètre l'objet *results* construit précédemment et qui renvoie un graphique similaire à celui renvoyé par `plot_learning_curves`. Cependant, les différentes itérations doivent être présente sur chaque graphique, ici les courbes d'entraînement pour chaque learning rate, avec la bonne légende pour chaque graphique.

**Consigne**: Lancer l'entraînement pour plus d'époques afin de comparer avec la fonction `show_results` les différences d'entraînement. Commenter.

Pour gagner du temps, on pourra augmenter le batch_size à 256 voire 528. Pour éviter de surcharger l'affichage, on peut utiliser le paramètre *verbose* de la méthode `fit` : s'il vaut 0 alors il n'y a aucun affichage.

## Pour continuer

Choisir une ou plusieurs pistes de recherche parmi les suivantes. Il est possible de choisir une autre direction, mais elle doit être validé auparavant.

1. L'initialisation des réseaux de neurones étant aléatoire, et la mise à jour des poids étant réalisées avec SGD, on ne peut pas considérer un exemple comme une généralité. Reproduire l'étude précédente en lançant plusieurs fois le même modèle pour être capable de générer un graphique avec des intervalles de confiance.
2. Nous avons vu en cours que l'initialisation des poids peut avoir un impact fort sur la suite de l'entraînement. En exploitant le paramètre `kernel_initializer` présent dans la définition de la couche [`Dense`](https://keras.io/api/layers/core_layers/dense/), proposer et réaliser une étude pour vérifier ou infirmer cela.
3. Les réseaux de neurones peuvent sur-apprendre. Il est important de pouvoir les régulariser. En exploitant le paramètre `kernel_regularizer` présent dans la définition de la couche [`Dense`](https://keras.io/api/layers/core_layers/dense/), proposer une étude pour visualiser son impact sur l'apprentissage.